In [19]:
import tkinter as tk
from tkinter import filedialog
from tkinter import PhotoImage
import csv
import requests
from bs4 import BeautifulSoup
import json
from pytube import Playlist
from pytube import YouTube
from tkinter import Tk, Canvas, Frame, BOTH, ttk
from tkinter import messagebox

def web_request(url):
    response = requests.get(url)
    # Create a BeautifulSoup object with the response content
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

def tkinter_open_file():
    csv_file = filedialog.askopenfilename(initialdir="/", title="Select File", filetypes=(("CSV files", "*.csv"), ("Text files", "*.txt"), ("All files", "*.*")))
    return csv_file

def tkinter_select_output_path():
    output_path = filedialog.askdirectory(initialdir="/", title="Select Output Path")
    return output_path

def download_youtube_video(yt_video_url, s_res):  
    # next update: bar progress status
    output_path = filedialog.askdirectory(initialdir="/", title="Select Output Path")
    try:
        youtube = YouTube(yt_video_url)  
        video = youtube.streams.filter(res=s_res).first()
        if video is None:
            # print(f"No video available for resolution: {s_res} - Skipping: {yt_video_url}")
            messagebox.showinfo(message = f"No video available for resolution: {s_res} - Skipping: {yt_video_url}")
        video.download(output_path,filename_prefix=s_res+'_')
        video_title = video.title
        # print(f"Downloaded: {yt_video_url} - Path: {output_path}")
        messagebox.showinfo(message = f"Downloaded: {yt_video_url} \n Path: {output_path} \n File:{video_title}")
    except Exception as e:
        # print(f"Error downloading {yt_video_url}: {str(e)}")
        messagebox.showinfo(message = f"Error downloading {yt_video_url}: {str(e)}")
    window.destroy()            

def download_youtube_videos_CSVfile(csv_file, m_res):
    # Next update add column "Resolution" in csv file in order to download in several resolution or different ones for each file/video 
    # Add summary at the process end in a message box whit "list name", "resolution" from download files
    output_path = filedialog.askdirectory(initialdir="/", title="Select Output Path")
    with open(csv_file, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row if present
        
        for row in csv_reader:
            video_url = row[0]
            try:
                youtube = YouTube(video_url)
                video = youtube.streams.filter(res=m_res).first()
                if video is None:
                    # print(f"No video available for resolution: {m_res} - Skipping: {video_url}")
                    messagebox.showinfo(message = f"No video available for resolution: {m_res} - Skipping: {video_url}")
                    continue
                video.download(output_path)
                video_title = video.title
                
                # csv_download_list = [(f"Downloaded: {video_url} - Path: {output_path}\n File:{video_title}")]
                
                
                # print(f"Downloaded: {video_url}")
                messagebox.showinfo(message = f"Downloaded: {video_url} \n Path: {output_path}\n File:{video_title}")
            except Exception as e:
                # print(f"Error downloading {video_url}: {str(e)}")
                messagebox.showinfo(message = f"Error downloading {video_url}: {str(e)}")
        window.destroy()

def download_youtube_playlist(yt_playlist_url, output_path, pl_resolution):
    # .........
    return 

def process_single_URL():
    yt_video_url = simple_url_entry.get()
    s_res = s_res_combobox.get()
    download_youtube_video(yt_video_url, s_res)
    return 

def process_csv_file():
    csv_file = filedialog.askopenfilename(initialdir="/", title="Select File", filetypes=(("CSV files", "*.csv"), ("Text files", "*.txt"), ("All files", "*.*")))
    m_res = m_res_combobox.get()
    download_youtube_videos_CSVfile(csv_file, m_res)
    return 

def paste_text():
    clipboard = root.clipboard_get() # Get the copied item from system clipboard
    url_frame_entry_url.insert(tk.END, clipboard) # Insert the item into the entry widget
    return

def show_menu(event):
    url_frame_entry_menu.post(event.x_root, event.y_root)


# def tkinter_interface():
# Create the main window
root = tk.Tk()
root.title("Download - YouTube Videos by Chaskas")
root.geometry("400x300")
root.maxsize(width= 400, height=300)
root.minsize(width= 400, height=300)
root.iconbitmap(False, 'down-red-icon.ico')

# Create a label for the options
root_label_options = tk.Label(root, text="Select an option:",font=("Arial", 12))
root_label_options.pack(anchor='nw')

url_frame = tk.Frame(root)
url_frame.pack()

# -----------------------------------"Single Video URL/ Playlist URL"-------------------------------------
# ------------------------------------------------------------------------------------
url_frame_box = tk.LabelFrame(url_frame, text = "Download Video from Single YT Link")
url_frame_box.grid(row= 0, column=0, padx=1, pady=1)    

# Create Entities
# URL frame (links)
url_frame_label_url = tk.Label(url_frame_box, text="Single Video URL / \n Playlist URL")
url_frame_label_url.grid(row= 0, column=0)

url_frame_entry_url = tk.Entry(url_frame_box)
url_frame_entry_url.grid(row= 1, column=0)
url_frame_entry_url.bind("<Button-3>", show_menu)

url_frame_entry_menu = tk.Menu(root, tearoff=False)
url_frame_entry_menu.add_command(label="Paste", command=paste_text)


url_frame_label_res = tk.Label(url_frame_box, text="Resolution")
url_frame_label_res.grid(row= 0, column=1)

url_frame_combobox_res = ttk.Combobox(url_frame_box, values=["144p", "240p", "360p", "480p", "720p", "1080p"],width=6)
url_frame_combobox_res.current(2)
url_frame_combobox_res.grid(row= 1, column=1)

# # Create a button to get the selected option
url_frame_singleURL = tk.Button(url_frame_box, text=" Process Single URL ", command=process_single_URL)
url_frame_singleURL.grid(row= 0, column=2)

url_frame_playlistURL = tk.Button(url_frame_box, text="Process Playlist URL", command=download_youtube_playlist)    
url_frame_playlistURL.grid(row= 1, column=2)


for widget in url_frame_box.winfo_children():
    widget.grid_configure(padx=10, pady=5)


"""
# # https://www.youtube.com/watch?v=ZxngoItDVt8

# # # ---------------------------------------Multiple links by CSV File-------------------
# # # ------------------------------------------------------------------------------------
# multiple_csv_frame = tk.Frame(root)
# multiple_csv_frame.pack()

# multiple_csv_frame = tk.LabelFrame(multiple_csv_frame, text = "Download Videos Multiple YT links from CSV file")
# multiple_csv_frame.grid(row= 0, column=0, padx=20, pady=20)    

# m_res_label = tk.Label(multiple_csv_frame, text="Average Resolution")
# m_res_combobox = ttk.Combobox(multiple_csv_frame, values=["144p", "240p", "360p", "480p", "720p", "1080p"],width=6)

# # Create a button for opening the file
# multiple_csv_button = tk.Button(multiple_csv_frame, text="Open and Process CSV File", command=process_csv_file)


# m_res_label.grid(row= 0, column=0)
# m_res_combobox.grid(row= 1, column=0)
# m_res_combobox.current(2)
# multiple_csv_button.grid(row= 1, column=1)



# for widget in multiple_csv_frame.winfo_children():
#     widget.grid_configure(padx=10, pady=5)
"""
"""
# # -----------------------------------PUBLIC URL Playlist------------------------------
# # ------------------------------------------------------------------------------------

# radio_playlist = tk.Label(window, text="PUBLIC Playlist URL ")
# radio_playlist.pack()




# # ------------------------------------------------------------------------------------
# # ------------------------------------------------------------------------------------

"""

# Run the main window's event loop
root.mainloop()

 


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\AlvaroDLC Work\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\AlvaroDLC Work\AppData\Local\Temp\ipykernel_23632\1184438931.py", line 97, in show_menu
    root_label_options.post(event.x_root, event.y_root)
    ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Label' object has no attribute 'post'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\AlvaroDLC Work\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\AlvaroDLC Work\AppData\Local\Temp\ipykernel_23632\1184438931.py", line 97, in show_menu
    root_label_options.post(event.x_root, event.y_root)
    ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Label' object has no attribute 'post'
Exception in Tkinter callbac